# Marketing (Facts) Models

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop queries for **mart (user orders)** models using DBT's `intermediate` models.

The version of these queries that does not use `intermediate` models is found in `from-staging/10_00_marketing_mart_users_orders_queries.ipynb`.

### User Order Facts (`fct_user_orders`)

#### Objective

Develop a model for the marketing team to perform the following
         
1. analyze user ordering behavior
2. identify the biggest customers so we can track their ordering activity over time.

#### Constraints

The result of this model should be aggregated at the user order level so it can be used by the marketing team in any other downstream model that is also aggregated at the user order level. Only user ID, state and aggregated columns should be included in this model.

#### Business Questions

1. Get the following order-related details for each user
   - first and last order date
   - total dollars spent on order
     - use this to determine which user is our biggest customer
   - number of orders
   - average number of products offered on the Greenery platform that are included in each order
   - average order size
     - average quantity of products across all orders

Include users who have not placed orders yet since we'll want to monitor how their purchasing behavior changes over time.

### Promo Facts (`fct_promo_orders`)

#### Objective

Develop a data model to facilitate querying data about the daily impact of promotions on orders.

#### Constraints

The result of this model should be shown daily since we want to track the impact of promotions over time.

#### Business Questions

1. Get the following daily order metrics for each promotion
   - (total) order size
   - (total) order (dollar) value
   - (average) number of unique products included in a single order
   - (average) order size (total quantity)
   - (total) shipping cost

Include orders in which users have redeemed a promocode since we can only quantify the impact of a promotion if it was included in an order.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to database

In [ ]:
%sql engine --alias connection

## Model

### `marts/marketing/fct_user_orders`

In [ ]:
%%sql
WITH orders_with_delivery_time AS (
    SELECT user_id,
           state_name,
           created_at,
           discount,
           order_total,
           num_unique_products,
           total_order_size,
           status,
           order_id,
           datediff(second, created_at, delivered_at) AS delivery_time_seconds
    FROM int_orders_joined_to_addresses_promos
),
user_order_summary AS (
    SELECT user_id,
           state_name,
           TO_DATE(MIN(created_at)) AS first_order_date,
           TO_DATE(MAX(created_at)) AS last_order_date,
           ZEROIFNULL(SUM(discount)) AS discount_value,
           -- get total dollar value of all orders by user, rounded to three
           -- decimal places
           ZEROIFNULL(ROUND(SUM(order_total), 2)) AS order_value,
           -- get average number of products in an order
           ZEROIFNULL(
               ROUND(AVG(num_unique_products))
           ) AS avg_num_unique_products,
           -- get average order size
           ZEROIFNULL(ROUND(AVG(total_order_size))) as avg_order_size,
           -- get total number of orders by user
           ZEROIFNULL(COUNT(order_id)) AS num_orders,
           -- get total number of delivered orders by user
           SUM(
               CASE WHEN status = 'delivered' THEN 1 ELSE 0 END
           ) AS num_orders_delivered,
           -- get total number of orders by user that are shipping
           SUM(
               CASE WHEN status = 'shipped' THEN 1 ELSE 0 END
           ) AS num_orders_shipping,
           -- get average delivery time for delivered orders by user
           ROUND(AVG(delivery_time_seconds)) AS avg_delivery_time_seconds
    FROM orders_with_delivery_time
    GROUP BY ALL
    ORDER BY order_value DESC
)
SELECT *
FROM user_order_summary

### `marts/marketing/fct_promo_orders`

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

In [ ]:
%%sql
WITH order_summary AS (
    SELECT order_id,
           TO_DATE(created_at) AS created_at_date,
           order_cost,
           promo_id,
           discount,
           shipping_cost,
           total_order_size,
           num_unique_products
    FROM int_orders_joined_to_addresses_promos
),
promo_order_summary AS (
    SELECT promo_id,
           created_at_date,
           COUNT(*) AS num_orders,
           SUM(discount) AS promo_discount,
           ROUND(SUM(shipping_cost), 2) AS shipping_cost,
           SUM(order_cost) AS order_cost,
           ROUND(AVG(num_unique_products)) AS avg_num_unique_products,
           ROUND(AVG(total_order_size)) AS avg_total_order_size
    FROM order_summary
    WHERE promo_id IS NOT NULL
    GROUP BY promo_id, created_at_date
    ORDER BY created_at_date ASC, promo_discount DESC
)
SELECT *
FROM promo_order_summary

## Disconnect

Close connection

In [ ]:
%sql --close connection

## Links

1. [Basic tutorial on `ROUND`](https://medium.com/@myemail.srinivas/round-function-in-snowflake-9c131baa97b5)